In [1]:
import warnings
warnings.simplefilter('ignore')
from pyedflib import EdfReader
from matplotlib import pyplot as plt
from tsfresh.feature_extraction import extract_features, MinimalFCParameters
import pandas as pd
import numpy as np

In [2]:
WINDOW_LENGTH=20             #Длина скользящего окна в секундах(нечётное значение)
PATH='data/'
CHANNELS_LIST='channels.xlsx'

In [3]:
class EmgFCPParameters(MinimalFCParameters):
    """
    Класс настройки списка извлекаемых признаков
    """
    def __init__(self):
        MinimalFCParameters.__init__(self)
        
        to_del=['length','variance','sum_values']
        to_add=['abs_energy','mean_abs_change','mean_change']
        
        for param in to_del:
            del self[param]
        
        for param in to_add:
            self[param]=None
            

In [4]:
def movie_window(file,chnum,wsec=WINDOW_LENGTH):
    """
    Функция скользящего окна заданной длительности, на выходе получаем numpy.array,
    содержащий оконные срезы для каждой секунды
    """
    frq=file.getSampleFrequency(chnum)
    record=file.readSignal(chnum)
    rlen=file.getNSamples()[chnum]
    
    rsec=rlen//frq
    record_windows=np.zeros((rsec-(wsec//2)*2,wsec*frq))
    
    for i,row in enumerate(record_windows):
        record_windows[i]=record[i*frq:(i+wsec)*frq]
    
    return record_windows

In [5]:
def to_tsfresh_DataFrame(record_windows,wsec=WINDOW_LENGTH):
    """
    Функция формирующая DataFrame для извлечения признаков с помощью tsfresh,
    из numpy.array поллученного movie_window
    """
    df=pd.DataFrame(record_windows)
    df=df.stack(level=0).reset_index()
    df.columns=['wtime_sec','samples','values']
    df['wtime_sec']=df['wtime_sec']+WINDOW_LENGTH//2+1
    return df

In [6]:
def get_params(records,params=EmgFCPParameters()):
    """
    Функция извлечения признаков из DataFrame, полученного с помощью to_tsfresh_DataFrame
    """
    return extract_features(records,
                            default_fc_parameters=params,
                            column_id='wtime_sec',
                            column_sort="samples")

In [7]:
def movie_window_params(filename,chnum,path=PATH,wsec=WINDOW_LENGTH):
    """
    """
    EDFData=EdfReader(path+filename)
    record_windows=movie_window(EDFData,chnum,wsec=WINDOW_LENGTH)
    dataf=to_tsfresh_DataFrame(record_windows)
    params=get_params(dataf)
    params['File']=filename
    params['Channel']=chnum
    EDFData._close()
    return params

In [8]:
def multifiles_params(channels=CHANNELS_LIST,path=PATH):
    channels_df=pd.read_excel(path+channels)
    params_df_list=[]
    
    for i,row in channels_df.iterrows():
        print('File '+str(row['File'])+', Channel '+str(row['Channel']))
        params_df_list.append(movie_window_params(str(row['File'])+'.edf',row['Channel']))
    
    params_df = pd.concat(params_df_list).reset_index()
    params_df.index.names=['Index']
    params_df.to_csv(PATH+'channels'+'_params.csv')
    
    return params_df

In [9]:
%%time
params_df_list=multifiles_params()

File 1, Channel 19


Feature Extraction: 100%|██████████| 10/10 [00:08<00:00,  1.19it/s]


File 2, Channel 7


Feature Extraction: 100%|██████████| 10/10 [00:09<00:00,  1.04it/s]


File 2, Channel 9


...

File 3, Channel 9


Feature Extraction: 100%|██████████| 10/10 [00:08<00:00,  1.25it/s]


File 4, Channel 15


Feature Extraction: 100%|██████████| 10/10 [00:09<00:00,  1.04it/s]


CPU times: user 3min 20s, sys: 1min 13s, total: 4min 34s
Wall time: 5min 19s


In [ ]:
len(params_df_list)

In [ ]:
params_df_list[0].head()

In [ ]:
result = pd.concat(params_df_list).reset_index()

In [ ]:
result.to_csv(PATH+'channels'+'_params.csv')

In [ ]:
result.index.names=['Index']